# Parquet konvertering-, partisjonering- og filtreringsverktøy for AIS-data

Dette verktøyet konverterer Parquet-filer til GeoParquet-format, partisjonerer på angitte kolonner og filtrerer på angitt dato og radius rundt Kristiansand sentrum. Data vises på tabell og kart.

Verktøyet er en løsningsmodell som bruker GeoPandas for følgende brukerhistorie:

"Som havnesjef i Kristiansand kommune ønsker jeg å vite hvilke skip som befant seg innen 6 km fra Kristiansand sentrum på en bestemt dato."

### Hvordan bruke dette verktøyet
1. Last opp dine geografiske datafiler til mappen `data/raw`
2. Kjør alle cellene i denne notebooken
3. De konverterte filene vil bli lagret i mappen `data/processed`

### Støttede filformater

- Parquet-filer med geografisk informasjon

In [5]:
import os
import re
import datetime

def find_newest_folder(folder_path, use_foldername=True):
    """
    Finner den nyeste mappen i en mappe basert på mappenavnet (dato) eller modifiseringstidspunkt.
    Leter etter mappenavn på formatet: hais_2024-12-24.snappy.parquet

    Args:
        folder_path (str): Stien til mappen hvor undermappene skal søkes
        use_foldername (bool): Om dato i mappenavn skal brukes (True) eller sist endret tid (False)

    Returns:
        tuple: (str: Stien til den nyeste mappen, str: Datoen som string i format 'YYYY-MM-DD',
               eller None hvis det ikke finnes mapper eller dato ikke ble funnet)
    """
    try:
        # Sjekk om mappen eksisterer
        if not os.path.exists(folder_path):
            print(f"Mappen {folder_path} eksisterer ikke")
            return None, None

        # Finn alle mapper i mappen (ikke filer)
        folders = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                  if os.path.isdir(os.path.join(folder_path, f))]

        if not folders:
            print(f"Ingen mapper funnet i {folder_path}")
            return None, None

        if use_foldername:
            # Finn den nyeste mappen basert på dato i mappenavnet
            newest_date = None
            newest_folder = None
            date_string = None

            for folder in folders:
                foldername = os.path.basename(folder)
                # Søk etter dato i formatet YYYY-MM-DD i mappenavnet
                date_match = re.search(r'(\d{4}-\d{2}-\d{2})', foldername)

                if date_match:
                    date_str = date_match.group(1)
                    try:
                        folder_date = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()

                        if newest_date is None or folder_date > newest_date:
                            newest_date = folder_date
                            newest_folder = folder
                            date_string = date_str
                    except ValueError:
                        # Hvis datoen ikke kan tolkes, hopp over denne mappen
                        continue

            if newest_folder:
                return newest_folder, date_string
            else:
                print("Ingen mapper med gyldig datoformat funnet. Bruker modifiseringstidspunkt i stedet.")
                # Fall tilbake til modifiseringstidspunkt
                newest_folder = max(folders, key=os.path.getmtime)
                # Ingen dato funnet i mappenavn, så vi returnerer None som date_string
                return newest_folder, None
        else:
            # Finn den nyeste mappen basert på modifiseringstidspunkt
            newest_folder = max(folders, key=os.path.getmtime)

            # Sjekk om vi kan finne en dato i mappenavnet til den nyeste mappen
            foldername = os.path.basename(newest_folder)
            date_match = re.search(r'(\d{4}-\d{2}-\d{2})', foldername)
            date_string = date_match.group(1) if date_match else None

            return newest_folder, date_string

    except Exception as e:
        print(f"Feil ved søk etter nyeste mappe: {str(e)}")
        return None, None

In [6]:
folder_path = "data/processed"
find_newest_folder(folder_path, use_foldername=True)

('data/processed/hais_2025-01-22.snappy.parquet', '2025-01-22')